# Problem Statement:

This data is for the purpose of bias correction of next-day maximum and minimum air temperatures forecast of the LDAPS model operated by the Korea Meteorological Administration over Seoul, South Korea. This data consists of summer data from 2013 to 2017. The input data is largely composed of the LDAPS model's next-day forecast data, in-situ maximum and minimum temperatures of present-day, and geographic auxiliary variables. There are two outputs (i.e. next-day maximum and minimum air temperatures) in this data. Hindcast validation was conducted for the period from 2015 to 2017.


Attribute Information:

For more information, read [Cho et al, 2020].
1. station - used weather station number: 1 to 25
2. Date - Present day: yyyy-mm-dd ('2013-06-30' to '2017-08-30')
3. Present_Tmax - Maximum air temperature between 0 and 21 h on the present day (Â°C): 20 to 37.6
4. Present_Tmin - Minimum air temperature between 0 and 21 h on the present day (Â°C): 11.3 to 29.9
5. LDAPS_RHmin - LDAPS model forecast of next-day minimum relative humidity (%): 19.8 to 98.5
6. LDAPS_RHmax - LDAPS model forecast of next-day maximum relative humidity (%): 58.9 to 100
7. LDAPS_Tmax_lapse - LDAPS model forecast of next-day maximum air temperature applied lapse rate (Â°C): 17.6 to 38.5
8. LDAPS_Tmin_lapse - LDAPS model forecast of next-day minimum air temperature applied lapse rate (Â°C): 14.3 to 29.6
9. LDAPS_WS - LDAPS model forecast of next-day average wind speed (m/s): 2.9 to 21.9
10. LDAPS_LH - LDAPS model forecast of next-day average latent heat flux (W/m2): -13.6 to 213.4
11. LDAPS_CC1 - LDAPS model forecast of next-day 1st 6-hour split average cloud cover (0-5 h) (%): 0 to 0.97
12. LDAPS_CC2 - LDAPS model forecast of next-day 2nd 6-hour split average cloud cover (6-11 h) (%): 0 to 0.97
13. LDAPS_CC3 - LDAPS model forecast of next-day 3rd 6-hour split average cloud cover (12-17 h) (%): 0 to 0.98
14. LDAPS_CC4 - LDAPS model forecast of next-day 4th 6-hour split average cloud cover (18-23 h) (%): 0 to 0.97
15. LDAPS_PPT1 - LDAPS model forecast of next-day 1st 6-hour split average precipitation (0-5 h) (%): 0 to 23.7
16. LDAPS_PPT2 - LDAPS model forecast of next-day 2nd 6-hour split average precipitation (6-11 h) (%): 0 to 21.6
17. LDAPS_PPT3 - LDAPS model forecast of next-day 3rd 6-hour split average precipitation (12-17 h) (%): 0 to 15.8
18. LDAPS_PPT4 - LDAPS model forecast of next-day 4th 6-hour split average precipitation (18-23 h) (%): 0 to 16.7
19. lat - Latitude (Â°): 37.456 to 37.645
20. lon - Longitude (Â°): 126.826 to 127.135
21. DEM - Elevation (m): 12.4 to 212.3
22. Slope - Slope (Â°): 0.1 to 5.2
23. Solar radiation - Daily incoming solar radiation (wh/m2): 4329.5 to 5992.9
24. Next_Tmax - The next-day maximum air temperature (Â°C): 17.4 to 38.9
25. Next_Tmin - The next-day minimum air temperature (Â°C): 11.3 to 29.8T

Please note that there are two target variables here: 

1) Next_Tmax: Next day maximum temperature

2) Next_Tmin: Next day  minimum temperature

In [ ]:
#Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.model_selection import train_test_split
from scipy.stats import zscore #to remove outliers
from scipy.stats import skew
import requests
import pandas_profiling
import io
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Importing dataset

In [ ]:
df = pd.read_csv("temperature.csv")

# eda

In [ ]:
df.head(10)

In [ ]:
df.shape # check the data dimension

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

## handle categorical data

In [ ]:
df[["day","month","year"]] = df["Date"].str.split("-", expand = True)

In [ ]:
df.drop(columns=["Date"], axis = 1, inplace= True)

In [ ]:
df

In [ ]:
df["day"] = df["day"].astype(float)
df["month"] = df["month"].astype(float)
df["year"] = df["year"].astype(float)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
#Checking the distribution of values of each column

In [ ]:
for col in df:
    print(col)
    
    plt.figure()
    sns.kdeplot(df[col], shade = True)
    plt.show()

In [ ]:
pre_profile = df.profile_report(title="temperature forecast")

In [ ]:
pre_profile

## check for outliers

In [ ]:
#remove outliers before skewness check and before x, y split

In [ ]:
df.boxplot(figsize=[20,8])
plt.subplots_adjust(bottom=0.25)
plt.show()

In [ ]:
#Removing outliers by z score

In [ ]:
from scipy.stats import zscore
z = np.abs(zscore(df))
new_df = df[(z<3).all(axis=1)]

In [ ]:
new_df.shape

In [ ]:
df.shape

In [ ]:
dataloss = ((7752-0)/7752)*100

In [ ]:
dataloss

In [ ]:
#Data loss is high, hence not dropping outliers

In [ ]:
df.isna().sum()

In [ ]:
df = df.fillna(value=df.mean())

In [ ]:
df.isna().sum()

In [ ]:
df.head()

## check co-relation

In [ ]:
#Arrange co-relation in descending order. Dropping columns should be the last option to prevent data loss.

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=[22,12])
cor = df.corr()
sns.heatmap(cor, annot = True)
plt.show()

In [ ]:
cor["Next_Tmin"].sort_values(ascending=False)

In [ ]:
df.columns

In [ ]:
columns = ['station', 'Present_Tmax', 'Present_Tmin', 'LDAPS_RHmin', 'LDAPS_RHmax',
       'LDAPS_Tmax_lapse', 'LDAPS_Tmin_lapse', 'LDAPS_WS', 'LDAPS_LH',
       'LDAPS_CC1', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4', 'LDAPS_PPT1',
       'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4', 'lat', 'lon', 'DEM', 'Slope',
       'Solar radiation', 'Next_Tmax', 'Next_Tmin', 'day', 'month', 'year']

## Predicting Tmax

## check for skewness

In [ ]:
x = df.drop('Next_Tmax',axis=1)
y = df['Next_Tmax']

In [ ]:
for col in df:
    print(col)
    print(skew(df[col]))
    
    plt.figure()
    sns.distplot(df[col])
    plt.show()

In [ ]:
x.skew() # check skewness

In [ ]:
from sklearn.preprocessing import power_transform
df_new = power_transform(x)

df_new = pd.DataFrame(df_new, columns = x.columns)

In [ ]:
df_new.skew()

In [ ]:
df_new

In [ ]:
x

In [ ]:
x = df_new

## finding the best random state

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.linear_model import LinearRegression
maxAccu=0
maxRS=0
for i in range(1,200):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.30, random_state = i)
    LR = LinearRegression()
    LR.fit(x_train, y_train)
    predLR = LR.predict(x_test)
    acc = r2_score(y_test, predLR)
    if acc > maxAccu:
        maxAccu = acc
        maxRS=i
print("Best accuracy is", maxAccu," on Random State ",maxRS)

## test train split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=124)

## regression

In [ ]:
#Linear Regression

In [ ]:
LR.fit(x_train, y_train)
predLR = LR.predict(x_test)
acc = r2_score(y_test, predLR)
acc
plt.scatter(y_test,predLR,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, predLR))
print('MSE:', metrics.mean_squared_error(y_test, predLR))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predLR)))
print('Variance:', metrics.explained_variance_score(y_test, predLR))
print('R2 Score:', r2_score(y_test, predLR))

## Cross-validation of the model

In [ ]:
from sklearn.model_selection import cross_val_score
for j in range(2,10):
    cv_score = cross_val_score(LR, x, y, cv = j)
    cv_mean = cv_score.mean()
    print(f"At cross fold {j} the cv score is {cv_mean} and accuracy score for training is {acc}")
    print("\n")

In [ ]:
#Since number of folds don't have much impact on the accuracy and cv_score, 
#cv = 5 is selected

## Regularization

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.linear_model import Lasso

parameters = {'alpha': [.0001, .001, .01, 1, 10], 'random_state': list(range(0,10))}
ls = Lasso()
clf = GridSearchCV(ls, parameters)
clf.fit(x_train, y_train)

print(clf.best_params_)

In [ ]:
ls = Lasso(alpha=0.0001, random_state= 0)
ls.fit(x_train, y_train)
ls.score(x_train, y_train)

pred_ls = ls.predict(x_test)

lss = r2_score(y_test, pred_ls)
lss

In [ ]:
#CatBoostRegressor

In [ ]:
from catboost import CatBoostRegressor
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=10,learning_rate=0.5,depth=2)
# Fit model
model.fit(x_train,y_train)
# Get predictions
preds = model.predict(x_test)

plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R2 Score', r2_score(y_test, preds))
print('Variance:',metrics.explained_variance_score(y_test, preds))

In [ ]:
#decisiontreeregressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(x_train,y_train)

# Predict
preds = regr_1.predict(x_test)

# Plot the results
plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('Variance:',metrics.explained_variance_score(y_test, preds))
print('R2 Score:',metrics.r2_score(y_test, preds))

## hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor()

gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=5,
                  n_jobs=1,
                  scoring='neg_mean_squared_error')

gs.fit(x_train, y_train)

print(gs.best_params_)
print(-gs.best_score_)

In [ ]:
new_model = DecisionTreeRegressor(max_depth=10,
                                  min_samples_split=20)
new_model.fit(x_train, y_train)
preds = new_model.predict(x_test)

# Plot the results
plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('Variance:',metrics.explained_variance_score(y_test, preds))
print('R2 Score:',metrics.r2_score(y_test, preds))

## Predicting Tmin

## check for skewness

In [ ]:
x = df.drop('Next_Tmin',axis=1)
y = df['Next_Tmin']

In [ ]:
x.skew() # check skewness

In [ ]:
from sklearn.preprocessing import power_transform
df_new = power_transform(x)

df_new = pd.DataFrame(df_new, columns = x.columns)

In [ ]:
df_new.skew()

In [ ]:
df_new

In [ ]:
x

In [ ]:
x = df_new

## finding the best random state

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.linear_model import LinearRegression
maxAccu=0
maxRS=0
for i in range(1,200):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.30, random_state = i)
    LR = LinearRegression()
    LR.fit(x_train, y_train)
    predLR = LR.predict(x_test)
    acc = r2_score(y_test, predLR)
    if acc > maxAccu:
        maxAccu = acc
        maxRS=i
print("Best accuracy is", maxAccu," on Random State ",maxRS)

## test train split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=36)

## regression

In [ ]:
#Linear Regression

In [ ]:
LR.fit(x_train, y_train)
predLR = LR.predict(x_test)
acc = r2_score(y_test, predLR)
acc
plt.scatter(y_test,predLR,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, predLR))
print('MSE:', metrics.mean_squared_error(y_test, predLR))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predLR)))
print('Variance:', metrics.explained_variance_score(y_test, predLR))
print('R2 Score:', r2_score(y_test, predLR))

## Cross-validation of the model

In [ ]:
from sklearn.model_selection import cross_val_score
for j in range(2,10):
    cv_score = cross_val_score(LR, x, y, cv = j)
    cv_mean = cv_score.mean()
    print(f"At cross fold {j} the cv score is {cv_mean} and accuracy score for training is {acc}")
    print("\n")

In [ ]:
#Since number of folds don't have much impact on the accuracy and cv_score, 
#cv = 5 is selected

## Regularization

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.linear_model import Lasso

parameters = {'alpha': [.0001, .001, .01, 1, 10], 'random_state': list(range(0,10))}
ls = Lasso()
clf = GridSearchCV(ls, parameters)
clf.fit(x_train, y_train)

print(clf.best_params_)

In [ ]:
ls = Lasso(alpha=0.0001, random_state= 0)
ls.fit(x_train, y_train)
ls.score(x_train, y_train)

pred_ls = ls.predict(x_test)

lss = r2_score(y_test, pred_ls)
lss

In [ ]:
#CatBoostRegressor

In [ ]:
from catboost import CatBoostRegressor
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=10,learning_rate=0.5,depth=2)
# Fit model
model.fit(x_train,y_train)
# Get predictions
preds = model.predict(x_test)

plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R2 Score', r2_score(y_test, preds))
print('Variance:',metrics.explained_variance_score(y_test, preds))

In [ ]:
#decisiontreeregressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(x_train,y_train)

# Predict
preds = regr_1.predict(x_test)

# Plot the results
plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('Variance:',metrics.explained_variance_score(y_test, preds))
print('R2 Score:',metrics.r2_score(y_test, preds))

## hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor()

gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=5,
                  n_jobs=1,
                  scoring='neg_mean_squared_error')

gs.fit(x_train, y_train)

print(gs.best_params_)
print(-gs.best_score_)

In [ ]:
new_model_Tmin = DecisionTreeRegressor(max_depth=9,
                                  min_samples_split=20)
new_model_Tmin.fit(x_train, y_train)
preds = new_model_Tmin.predict(x_test)

# Plot the results
plt.scatter(y_test,preds,color='b')
plt.plot(y_test,y_test, color='r')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

print('MAE:', metrics.mean_absolute_error(y_test, preds))
print('MSE:', metrics.mean_squared_error(y_test, preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('Variance:',metrics.explained_variance_score(y_test, preds))
print('R2 Score:',metrics.r2_score(y_test, preds))

In [ ]:
import joblib
joblib.dump(new_model, "Tmaxmodel.pkl") 
joblib.dump(new_model_Tmin, "Tminmodel.pkl")